# IMPORTS

In [2]:
import pandas as pd

import my_pickle as mp
import my_split as ms
import my_dataframes as md

from importlib import reload

In [14]:
reload(md)

<module 'my_dataframes' from '/Users/gandalf/Documents/Galvanize/MatchingService/my_dataframes.py'>

# READ IN

In [15]:
# read in message data
# message_df = md.get_message_df()
# convo_df = md.get_conversation_df(message_df)
# last_df = md.get_lastmessage_df()
user_df = md.get_user_data()

created user dataframe


In [20]:
user_df.onboarded.sum()
len(user_df)

35343

# ECT SPLIT

In [5]:
# find the split!
cutoff = ms.ect_find_split(convo_df, .7)
print("split on:         {}".format(pd.to_datetime(cutoff*1000000)))

# do the split!
safe_convo_df = ms.ect_make_split(convo_df, cutoff)
print("most recent data: {}".format(pd.to_datetime(safe_convo_df.timestamp.max()*1000000)))

split on:         2017-10-02 01:35:58.644000
most recent data: 2017-10-02 01:35:58.644000


# REMOVE ROWS FROM CONVOS WITH MISSING UIDS

In [ ]:
safe_convo_df = md.remove_bad_uids(safe_convo_df, user_df)  

# PICKLE EVERYTHING!

https://www.youtube.com/watch?v=yYey8ntlK_E

In [10]:
mp.pickle_it(user_df,'data_user')
# mp.pickle_it(message_df,'message_df')  NOT ECT SAFE!
mp.pickle_it(safe_convo_df,'data_convo')  # make sure pickling safe stuff
# pickle_it(lastmessage_df,'lastmessage_df') NOT ECT SAFE!

In [11]:
print(len(user_df))

35343
